In [2]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
SEED = 42
def cv_loop_acc(X, y, model, N):
    mean_acc = 0.0
    for i in range(0,N,1):
        X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(
                                       X, y, test_size=1.0/float(N), 
                                       random_state = i*SEED)
        model.fit(X_train, y_train)
        preds = model.predict(X_cv)
        acc = metrics.accuracy_score(y_cv, preds)
        #print acc
        mean_acc += acc
    return mean_acc/N

In [4]:
SEED = 42
def cv_loop_auc(X, y, model, N):
    mean_auc = 0.0
    for i in range(0,N,1):
        X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(
                                       X, y, test_size=1.0/float(N), 
                                       random_state = i*SEED)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_cv)[:,1]
        auc = metrics.roc_auc_score(y_cv, preds)
        #print auc
        mean_auc += auc
    return mean_auc/N

In [5]:
SEED = 42
def latest_day_loop_acc(X_train,y_train,X_test,y_test,model):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = metrics.accuracy_score(y_test, preds)
    return acc

In [6]:
def read_csv(day_trade):
    data_up = []
    data_down = []
    path = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/Train_Test_Data/ML_data_2014'
    for j,i in enumerate(day_trade):
        for k in range(0,len(i),1):
            path_up = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'UP' + '.csv'
            path_down = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'DOWN' + '.csv'
            data_up.append(pd.read_csv(path_up))
            data_down.append(pd.read_csv(path_down))
            #print path_down
    return data_up,data_down

In [7]:
day_trade = [[2,3,6,7,8,10,13,14,15,16,17,20,21,22,23,24,27,28,29,30],\
             [7,10,11,12,13,17,18,19,21,24,25,26,27],\
             [3,4,5,6,7,10,11,13,14,17,18,19,20,24,25,26,27,31]]

In [8]:
data_2014_up, data_2014_down = read_csv(day_trade)

In [25]:
latest_min = 60 * 30
pred_sec = 10
Accuracy_Mean_Day = []
Accuracy_Std_Day = []
Feature_Importance = []
Feature_Importance_Day_one = []
Feature_Importance_Day_two = []
Feature_Importance_Day_three = []
Feature_Importance_Day_four = []
Feature_Importance_Day_five = []


for day in range(0,1,1):
    Accuracy = []
    Feature_Importance_one = []
    Feature_Importance_two = []
    Feature_Importance_three = []
    Feature_Importance_four = []
    Feature_Importance_five = []
    for i in range(0,9000-latest_min-pred_sec,pred_sec):

        print '----------------------------Rolling Window Time = %s----------------------------'%(i/5.0)
        data_train = data_2014_up[day][i:i+latest_min]
        X_train = data_train.drop(["0"],axis=1)
        y_train = data_train['0']

        data_test = data_2014_up[day][i+latest_min:i+latest_min+pred_sec]
        X_test = data_test.drop(["0"],axis=1)
        y_test = data_test['0']

        # hyper-parameter
        num_trees = [10]
        max_depth = [10]
        criterion = ['entropy']
        min_samples_leaf = [5]
        min_samples_split = [2]
        max_features = [None]

        # cv
        model_grid = {'max_features':max_features,'n_estimators':num_trees,'max_depth':max_depth,\
                      'min_samples_split':min_samples_split,'criterion':criterion,\
                      'min_samples_leaf':min_samples_leaf}
        model = RandomForestClassifier(random_state = 0)
        Grid = GridSearchCV(model, model_grid, cv = 5)
        %time Grid.fit(X_train,y_train) 
        #print Grid.best_params_
        #print 'CV Best Score = %s'%(Grid.best_score_)

        # training & testing
        model.set_params(**Grid.best_params_)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        acc = metrics.accuracy_score(y_test, predictions)
        #print 'Accuracy = %s'%(acc)
        Accuracy.append(acc)
        feature_imp = dict(zip([i for i in range(0,64,1)],model.feature_importances_))
        
        Top_five = sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:5]
        print 'Top five importance features = %s'%(Top_five)
        
        Feature_Importance_one.append(Top_five[0][0])
        Feature_Importance_two.append(Top_five[1][0])
        Feature_Importance_three.append(Top_five[2][0])
        Feature_Importance_four.append(Top_five[3][0])
        Feature_Importance_D_five.append(Top_five[4][0])
        
    print 'Day = %s, mean = %s, std = %s'%(day,np.mean(Accuracy),np.std(Accuracy))
    Accuracy_Mean_Day.append(Accuracy)
    Accuracy_Std_Day.append(Accuracy)
    
    

----------------------------Rolling Window Time = 0.0----------------------------
CPU times: user 1.23 s, sys: 0 ns, total: 1.23 s
Wall time: 1.79 s
Top five importance features = [(26, 0.2756067374499116), (28, 0.19572412669985756), (30, 0.064353057612475117), (47, 0.061136974638440811), (31, 0.055952665806685717)]
----------------------------Rolling Window Time = 2.0----------------------------
CPU times: user 1.1 s, sys: 0 ns, total: 1.1 s
Wall time: 1.13 s
Top five importance features = [(26, 0.26690982524235302), (28, 0.14802233594031858), (31, 0.068699646636591796), (25, 0.059425792959110699), (47, 0.04267486241248182)]
----------------------------Rolling Window Time = 4.0----------------------------
CPU times: user 1.11 s, sys: 1.07 ms, total: 1.11 s
Wall time: 1.15 s
Top five importance features = [(26, 0.34947897259078509), (28, 0.1190503257994556), (31, 0.072079167291345739), (25, 0.046921971059614917), (5, 0.043369071232071774)]
----------------------------Rolling Window Tim

KeyboardInterrupt: 

In [156]:
from sklearn.grid_search import (GridSearchCV, RandomizedSearchCV)
from sklearn import (metrics, cross_validation, linear_model, preprocessing)
from sklearn import svm, datasets

In [44]:
i = 0
day = 0
latest_min = 60 * 30
pred_sec = 5

data_train = data_2014_up[day][i:i+latest_min]
X_train = data_train.drop(["0"],axis=1)
y_train = data_train['0']
data_test = data_2014_up[day][i+latest_min:i+latest_min+pred_sec]
X_test = data_test.drop(["0"],axis=1)
y_test = data_test['0']

In [84]:
turned_parameters

[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernal': ['rbf']},
 {'C': [1, 10, 100, 1000], 'kernal': ['linear']}]

In [234]:
turned_parameters = [{'kernel':['rbf'],'gamma':[1e-3,1e-4],'C':[1,10,100,1000]},
                     {'kernel':['linear'],'C':[1,10,100,1000]},
                     {'kernel':['poly'],}]

In [235]:
svr = svm.SVC(probability=True)

In [236]:
Grid = GridSearchCV(svr,turned_parameters,cv=5)

In [237]:
iris = datasets.load_iris()

In [238]:
Grid

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}, {'kernel': ['polynomial'], 'C': [1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [239]:
Grid.fit(iris.data,iris.target)

ValueError: 'polynomial' is not in list

In [ ]:
Grid.best_params_

In [200]:
Grid.predict(iris.data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [213]:
Grid = GridSearchCV(SVC(C=1),turned_parameters,cv=5)#,scoring='precision')

In [214]:
Grid.estimator.get_params

<bound method SVC.get_params of SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)>

In [215]:
%time Grid.fit(X_train,y_train)

CPU times: user 11.4 s, sys: 0 ns, total: 11.4 s
Wall time: 12.2 s


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [221]:
Grid.best_params_,Grid.best_score_,Grid.grid_scores_

({'C': 10, 'kernel': 'linear'},
 0.8111111111111111,
 [mean: 0.72500, std: 0.08540, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.001},
  mean: 0.69889, std: 0.00778, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001},
  mean: 0.78167, std: 0.13956, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001},
  mean: 0.73389, std: 0.08836, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001},
  mean: 0.79667, std: 0.16202, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.001},
  mean: 0.80500, std: 0.12103, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.0001},
  mean: 0.78722, std: 0.11947, params: {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001},
  mean: 0.80833, std: 0.14398, params: {'kernel': 'rbf', 'C': 1000, 'gamma': 0.0001},
  mean: 0.80333, std: 0.10952, params: {'kernel': 'linear', 'C': 1},
  mean: 0.81111, std: 0.09755, params: {'kernel': 'linear', 'C': 10},
  mean: 0.79111, std: 0.10890, params: {'kernel': 'linear', 'C': 100},
  mean: 0.78444, std: 0.10786, params: {'kernel': 'linear', 'C': 

In [217]:
metrics.accuracy_score(Grid.predict(X_test),y_test)

1.0